In [ ]:
# author: Vladimir Osin 

In [1]:
import os
import torch
import torch.nn            as nn
import torch.optim         as optim
import torch.nn.functional as F
import mlflow              as mf
import mlflow.pytorch
import shutil

from torch.utils.data import DataLoader
from torchvision      import datasets, transforms
from torch.autograd   import Variable

c:\users\resol\miniconda3\envs\mlflow_tutorial\lib\site-packages\torchvision\datasets\lsun.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [6]:
mf.set_tracking_uri("http://127.0.0.1:5000")

In [7]:
experiment_id = mf.set_experiment("Models - Serving")

INFO: 'Models - Serving' does not exist. Creating a new experiment


In [8]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [10]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from the URL
csv_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x  = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y  = test[["quality"]]

alpha    =  0.5
l1_ratio =  0.5

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [ ]:
!mlflow models serve -m "file:///C:\\Users\\resol\\OneDrive - Philips Lighting\\code\\mlflow_tutorial\\artifact_store\/4/207b226c40374acfafd464300f134f63/artifacts/model" -p 1234